<a href="https://colab.research.google.com/github/jasoriya/Mercari-Price-Suggestion-Challenge/blob/master/MPS_Data_Preprocessing_%26_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Pre-requisite libraries to install on Google Colab before running the code

If you are installing any dependent libraries, please specify here as pre-req 
to install before running the notebook


1.   scikit-learn (upgrade to v0.21.3)
2.   textstat (v0.5.6)


In [2]:
#run to install pre-req libraries
!pip install textstat
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.21.3)


In [3]:
from datetime import datetime 
start_real = datetime.now()
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydot_ng as pydot
from textstat import textstat
import dask.dataframe as dd
from IPython.display import display

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import plot_model
from keras import backend as K
from nltk.corpus import stopwords
import math
# set seed
np.random.seed(123)

Using TensorFlow backend.


# 1. Data Preprocesing

### Get the input folder of the dataset

In [4]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/content/gdrive/My Drive/DMT/mercari-price-suggestion-challenge"))
input_folder = '/content/gdrive/My Drive/DMT/mercari-price-suggestion-challenge/'

# Any results you write to the current directory are saved as output.

['sample_submission.csv.7z', 'sample_submission_stg2.csv', 'train.tsv', 'test.tsv', 'train.pkl', 'model.best.weights.hdf5', 'train_word2vec.pkl', 'model.png', 'model_word2vec.best.weights.hdf5', 'train_xgboost.pkl', 'dumps']


## 1.1 Load the Data

In [5]:
train_data = pd.read_csv(input_folder + 'train.tsv', sep='\t')
# test_data = pd.read_csv(input_folder + 'test.tsv', sep='\t')
print("train shape",np.shape(train_data))
# print("test shape",np.shape(test_data))
train_data.head()

train shape (1482535, 8)


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


In [7]:
train_data.describe()

,train_id,item_condition_id,price,shipping
count,1.482535e+06,1.482535e+06,1.482535e+06,1.482535e+06
mean,7.412670e+05,1.907380e+00,2.673752e+01,4.472744e-01
std,4.279711e+05,9.031586e-01,3.858607e+01,4.972124e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,3.706335e+05,1.000000e+00,1.000000e+01,0.000000e+00
50%,7.412670e+05,2.000000e+00,1.700000e+01,0.000000e+00
75%,1.111900e+06,3.000000e+00,2.900000e+01,1.000000e+00
max,1.482534e+06,5.000000e+00,2.009000e+03,1.000000e+00


##  1.2 Data Cleaning

In [8]:
train_data.price.describe().apply(lambda x: format(x, 'f'))

count    1482535.000000
mean          26.737516
std           38.586066
min            0.000000
25%           10.000000
50%           17.000000
75%           29.000000
max         2009.000000
Name: price, dtype: object

We can gather that the 75% of the products are price $29 or less.

As per Mercari policy, it allowed product pricing between `$3-$2000` when this dataset was released in 2017. So the datapoints out of that range seem to be incorrect. Removing them helps the models.

In [9]:
# Remove low prices, anything below 3
train_data = train_data.drop(train_data[(train_data.price < 3.0)].index)
train_data.shape

(1481661, 8)

In [10]:
train_data = train_data.drop(train_data[(train_data.price > 2000)].index)
train_data.shape

(1481658, 8)

In [11]:
train_data.price.describe().apply(lambda x: format(x, 'f'))

count    1481658.000000
mean          26.749280
std           38.489080
min            3.000000
25%           10.000000
50%           17.000000
75%           29.000000
max         2000.000000
Name: price, dtype: object

Now, the minimum has changed from `$0 to $3` but the rest of quantiles remain same. 

#### Handling Missing Data

In [12]:
# Check for missing values
train_data.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6314
brand_name           632336
price                     0
shipping                  0
item_description          4
dtype: int64

In [13]:
# Get the percentage of missing data for each column.
train_data.isnull().sum().div(len(train_data)*.01, axis='rows').apply(lambda x: format(x, 'f'))

train_id              0.000000
name                  0.000000
item_condition_id     0.000000
category_name         0.426144
brand_name           42.677595
price                 0.000000
shipping              0.000000
item_description      0.000270
dtype: object

The variables brand_name has a substantial amount of missing data. 

In [14]:
df_with_no_desc = train_data[train_data.item_description.str.contains(pat='no description yet', case=False, na=False, regex=False)]
print("Total {} datapoints have 'No description yet' as their  description.".format(str(len(df_with_no_desc))))
display(df_with_no_desc.head())
del df_with_no_desc

Total 82623 datapoints have 'No description yet' as their  description.


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
13,13,Sharpener and eraser,1,Other/Office supplies/School Supplies,Scholastic,4.0,1,No description yet
31,31,"New Lularoe OS leggings, stars",1,"Women/Athletic Apparel/Pants, Tights, Leggings",NaN,50.0,1,No description yet
90,90,Jewel holder,2,Home/Storage & Organization/Jewelry Boxes & Or...,Target,7.0,1,No description yet
117,117,Eyeko liquid eyeliner,1,Beauty/Makeup/Eyes,NaN,5.0,1,No description yet


We will consider these datapoints as NaN for the purpose of handling missing values. 

In [15]:
def fill_missing_data(data):
    data.category_name.fillna(value = "unk_cat1/unk_cat2/unk_cat3", inplace = True)
    data.item_description.fillna(value = "unk_description", inplace = True)
    data.loc[data.item_description.str.contains(pat='no description yet', case=False, na=False, regex=False), 'item_description'] = "unk_description"
    return data

train_data = fill_missing_data(train_data)
display(train_data.head())

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,unk_description
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


We will handle the missing values present in *brand_name* column at the feature engineering stage below. This is done because replacing 42% of datapoints with 'unk_brand' value will not be optimal for a ML model

##  1.3 Data Wrangling

### Feature Engineering

#### Create new feature '*name*' by concatenating existing '*name*' and '*brand_name*' columns.

In [16]:
train_data['name'] = train_data['name'].fillna('') + ' ' + train_data['brand_name'].fillna('')
train_data.drop('brand_name', axis=1, inplace=True)
display(train_data.head())

,train_id,name,item_condition_id,category_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,10.0,1,unk_description
1,1,Razer BlackWidow Chroma Keyboard Razer,3,Electronics/Computers & Tablets/Components & P...,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse Target,1,Women/Tops & Blouses/Blouse,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,44.0,0,Complete with certificate of authenticity


#### Strip extra whitespaces

In [17]:
def remove_whitespace( x ):
    if isinstance( x, str ):
        return x.strip()
    else:
        return x
print("BEFORE: No. of unique values\n")
display(train_data.nunique())
train_data = train_data.applymap(remove_whitespace)
print("\nAFTER: No. of unique values\n")
display(train_data.nunique())

BEFORE: No. of unique values



train_id             1481658
name                 1265455
item_condition_id          5
category_name           1288
price                    824
shipping                   2
item_description     1280483
dtype: int64


AFTER: No. of unique values



train_id             1481658
name                 1265388
item_condition_id          5
category_name           1288
price                    824
shipping                   2
item_description     1280483
dtype: int64

#### Standardizing the target variable price


In [18]:
# Standardize target variable price
from sklearn.preprocessing import StandardScaler
scalar=StandardScaler()
scalar.fit(train_data.price.values.reshape(-1,1))
joblib.dump(scalar, input_folder + 'dumps/scaler.pkl') # save the scalar model 
train_data['price'] = scalar.transform(train_data.price.values.reshape(-1,1))
display(train_data.price.head())

0   -0.435170
1    0.656049
2   -0.435170
3    0.214365
4    0.448198
Name: price, dtype: float64

#### Feature extraction from item_description



##### Calculate readability score from the item_description attribute.

In [19]:
%%time
ddata = dd.from_pandas(train_data, npartitions=2)
train_data['item_description_readability_score'] = ddata['item_description'].apply(lambda x: textstat.gunning_fog(x) if x is not '' and x.lower()!='unk_description' else 99, meta=(None, 'float64')).compute(scheduler='multiprocessing')
display(train_data[['item_description', 'item_description_readability_score']].head())

,item_description,item_description_readability_score
0,unk_description,99.00
1,This keyboard is in great condition and works ...,8.13
2,Adorable top with a hint of lace and a key hol...,5.80
3,New with tags. Leather horses. Retail for [rm]...,2.12
4,Complete with certificate of authenticity,18.00


CPU times: user 18.9 s, sys: 2.47 s, total: 21.4 s
Wall time: 2min 59s


##### Calculate word counts in 'item_description' and 'name' columns 

In [20]:
%%time
# get name and description lengths
def wordCount(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0
train_data['desc_len'] = train_data['item_description'].apply(lambda x: wordCount(x))
train_data['name_len'] = train_data['name'].apply(lambda x: wordCount(x))
display(train_data.head())

,train_id,name,item_condition_id,category_name,price,shipping,item_description,item_description_readability_score,desc_len,name_len
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,-0.435170,1,unk_description,99.00,1,7
1,1,Razer BlackWidow Chroma Keyboard Razer,3,Electronics/Computers & Tablets/Components & P...,0.656049,0,This keyboard is in great condition and works ...,8.13,36,5
2,2,AVA-VIV Blouse Target,1,Women/Tops & Blouses/Blouse,-0.435170,1,Adorable top with a hint of lace and a key hol...,5.80,29,3
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,0.214365,1,New with tags. Leather horses. Retail for [rm]...,2.12,32,3
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,0.448198,0,Complete with certificate of authenticity,18.00,5,4


CPU times: user 8.21 s, sys: 167 µs, total: 8.21 s
Wall time: 8.2 s


In [29]:
train_data[['name_len', 'desc_len', 'price']].corr(method='pearson')

,name_len,desc_len,price
name_len,1.000000,0.148235,0.074803
desc_len,0.148235,1.000000,0.048929
price,0.074803,0.048929,1.000000


**Important observation**:

The length of the description i.e. the raw number of words used, does have a slight positive correlation with price. 

These lengths might boost the model performance.

#### Handling categorical variables

The columns name, item_condition_id, category_name, brand_name have categorical values. Categorical values can be converted to One Hot Encoding to feed into a machine learning model. The column has fixed 5 unique values and we will transform that using OHE. 

The other remaining variables do not have fixed unique values, that can be problematic in OHE if the test dataset has new unique values/categories. These other variables alo have a large number of unique values, and this will create a very sparse matrix. Due to these problems, we will not transform them to OHE for now.

In [21]:
#One-hot-encoding for item_condition_id
train_data = pd.concat((train_data, pd.get_dummies(data=train_data.item_condition_id, prefix='item_condition_id')), axis=1)
display(train_data.head())

,train_id,name,item_condition_id,category_name,price,shipping,item_description,item_description_readability_score,desc_len,name_len,item_condition_id_1,item_condition_id_2,item_condition_id_3,item_condition_id_4,item_condition_id_5
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,-0.435170,1,unk_description,99.00,1,7,0,0,1,0,0
1,1,Razer BlackWidow Chroma Keyboard Razer,3,Electronics/Computers & Tablets/Components & P...,0.656049,0,This keyboard is in great condition and works ...,8.13,36,5,0,0,1,0,0
2,2,AVA-VIV Blouse Target,1,Women/Tops & Blouses/Blouse,-0.435170,1,Adorable top with a hint of lace and a key hol...,5.80,29,3,1,0,0,0,0
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,0.214365,1,New with tags. Leather horses. Retail for [rm]...,2.12,32,3,1,0,0,0,0
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,0.448198,0,Complete with certificate of authenticity,18.00,5,4,1,0,0,0,0


##### Split the category column

In [22]:
print("Total datapoints: {}\nNo. of datapoints that have 3 sub categories: {}".format(len(train_data), len(train_data.category_name.apply(lambda x: len(x.split('/'))) == 3)))

Total datapoints: 1481658
No. of datapoints that have 3 sub categories: 1481658


Every category has 3 subparts. We can split them into individual subparts. We split the category_name into 3 parts. Our models can get more information this way.

In [23]:
train_data = pd.concat([train_data, train_data.category_name.str.split('/', n = 2, expand=True).rename({0: 'cat_1', 1: 'cat_2', 2: 'cat_3'}, axis=1)], axis=1)
display(train_data.head())

,train_id,name,item_condition_id,category_name,price,shipping,item_description,item_description_readability_score,desc_len,name_len,item_condition_id_1,item_condition_id_2,item_condition_id_3,item_condition_id_4,item_condition_id_5,cat_1,cat_2,cat_3
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,-0.435170,1,unk_description,99.00,1,7,0,0,1,0,0,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard Razer,3,Electronics/Computers & Tablets/Components & P...,0.656049,0,This keyboard is in great condition and works ...,8.13,36,5,0,0,1,0,0,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse Target,1,Women/Tops & Blouses/Blouse,-0.435170,1,Adorable top with a hint of lace and a key hol...,5.80,29,3,1,0,0,0,0,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,0.214365,1,New with tags. Leather horses. Retail for [rm]...,2.12,32,3,1,0,0,0,0,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,0.448198,0,Complete with certificate of authenticity,18.00,5,4,1,0,0,0,0,Women,Jewelry,Necklaces


#### Prepare Train, Validation and Test sets

We will split train, validation and test sets in 90:5:5 ratio

In [30]:
#Split data into train set first and then split the remaining set into validation and test set.

train_df, remaining_df = train_test_split(train_data, random_state=42, train_size=0.9)
validation_df, test_df = train_test_split(remaining_df, random_state=42, train_size=0.5)
print("Datapoints in train, validation and test set: {}, {}, {}".format(len(train_df), len(validation_df), len(test_df)))

Datapoints in train, validation and test set: 1333492, 74083, 74083


In [31]:
train_df.head()

,train_id,name,item_condition_id,category_name,price,shipping,item_description,item_description_readability_score,desc_len,name_len,item_condition_id_1,item_condition_id_2,item_condition_id_3,item_condition_id_4,item_condition_id_5,cat_1,cat_2,cat_3
958469,958469,US soccer Nike jersey Nike,2,Men/Athletic Apparel/Jerseys,-0.305263,1,I found this jersey recently and it's honestly...,6.34,47,5,0,1,0,0,0,Men,Athletic Apparel,Jerseys
693778,693778,Men's levis 38 32 Levi's®,3,"Men/Jeans/Classic, Straight Leg",-0.227319,0,Great condition mens straight leg jeans,9.07,6,5,0,0,1,0,0,Men,Jeans,"Classic, Straight Leg"
210050,210050,Nike running hoodie Nike,2,Women/Sweaters/Hooded,-0.045449,1,White size medium dri fit running hoodie. Exce...,5.84,22,4,0,1,0,0,0,Women,Sweaters,Hooded
447294,447294,betsy johson purse Betsey Johnson,1,Women/Women's Handbags/Shoulder Bag,-0.071430,1,Brand new!! never used perfect condition,9.07,6,5,1,0,0,0,0,Women,Women's Handbags,Shoulder Bag
1480041,1480041,ORGANIC! EXTREME CLEANING & WHITENING!!,1,Beauty/Bath & Body/Cleansers,-0.357226,1,BLACK is the NEW WHITE! Our Activated Charcoal...,11.99,120,5,1,0,0,0,0,Beauty,Bath & Body,Cleansers


#### Save dataset using pickle 

In [0]:
#save the dataset using pickle
train_data.to_pickle(input_folder + 'dumps/whole_data.pkl') 
train_df.to_pickle(input_folder + 'dumps/train_data.pkl')
validation_df.to_pickle(input_folder + 'dumps/validation_data.pkl')
test_df.to_pickle(input_folder + 'dumps/test_data.pkl')